In [ ]:
from nb_utils import rescale_plot, mtick, md, run_simulation,\
                     getSimulationData, get_pool_agent, get_arb_env,\
                     get_LT_LP_Binance_data, pd, plt, np, ASSET_PRICE_INDEX,\
                     datetime, get_binance_month_data, plot_impact_curves,\
                     runOneSimulation, getOneSimulationData, plot_one_sim_result

In [ ]:
%load_ext autoreload
%autoreload 2

# Load data

In [ ]:
LPdata      = pd.read_pickle("data/LPdata.pkl")
LTdata      = pd.read_pickle("data/LTdata.pkl")
BinanceData = pd.read_pickle("data/BinanceData.pkl")

# Wealth distribution:
### CQV / Uniswap : 100% arbitrageurs

In [ ]:
def eta_func_bid(y, Delta, L):
    if y-Delta==0: return -L
    if y-Delta>0: return L * Delta  / (y - Delta)
    if y-Delta<0: return -L * Delta / (y - Delta)

def eta_func_ask(y, Delta, L):
    if y+Delta==0: return -L
    if y+Delta>0: return L * Delta  / (y + Delta)
    if y+Delta<0: return -L * Delta / (y + Delta)

In [ ]:
############################
# LP strategy parameters
############################
jump_size_L            = 0.2
phi                    = 1e-2
alpha                  = 1e-2
fill_exponent          = 0.2
initial_inventory_pool = 0

############################
# Pool liquidity parameters
############################
target_inventory       = initial_inventory_pool
min_inventory_pool     = initial_inventory_pool - 500.
max_inventory_pool     = initial_inventory_pool + 500.

############################
# Others
############################
max_depth              = 0
seed                   = 200
num_trajectories       = 1
verbose                = False

In [ ]:
from IPython.display import clear_output

np.random.seed(seed)
dict_results  = {}
allData_ = None
trade_dates = list(BinanceData.resample('d').last().dropna().index)

for td in trade_dates[20:]:
    from_dt_is = td - datetime.timedelta(minutes = 60)
    from_dt = td
    to_dt   = td + datetime.timedelta(minutes = 30)
    end_of_day = False
    
    while not end_of_day:
        trade_date    = str(td).split(' ')[0]
        from_datetime = str(from_dt)
        to_datetime   = str(to_dt)
        from_dt_insample = str(from_dt_is)
        clear_output(wait=True)
        
        # Some verbose
        if len(dict_results) > 0:
            pnls = [dict_results[k].LPWealth.iloc[-1] / (dict_results[k].pool_inv_y.abs().mean() * dict_results[k].Z.iloc[0]) for k in dict_results.keys()]
            print('Number of points : ', len(pnls))
            print('Last pnl : ', pnls[-1])
            print('Pnls:', np.mean(pnls), ' / ', np.std(pnls))
            print('\n')

        print('Trading from ', from_datetime, ' to ', to_datetime)
        
        try:
            ############################################
            # load data
            ############################################
            oneDayLTdata, oneDayLPdata, oneDaybinanceLTdata,\
            fill_exponents, pool_sizes, hist_prices,\
            initial_convexity, trade_sizes, bothPrices = get_LT_LP_Binance_data(LTdata, LPdata, BinanceData, trade_date, 
                                                                                from_datetime, to_datetime)
            bothPrices.columns = ['Binance', 'Uniswap']


            in_sample_data = BinanceData[ ((BinanceData.index<=from_datetime) &
                                            (BinanceData.index>from_dt_insample))]

            ############################################
            # estimate in-sample parameters
            ############################################
            initial_price          = hist_prices[0]
            n_steps                = len(oneDayLTdata)
            arrival_rate           = trade_sizes.mean() / (in_sample_data.reset_index().time.diff(1).mean().seconds/60 + in_sample_data.reset_index().time.diff(1).mean().microseconds // 1000000/60)
            unit_size              = 1 #oneDaybinanceLTdata.qty.mean() #10 #(max_inventory_pool - min_inventory_pool) / 100 # matrix of size 1000
            terminal_time          = (bothPrices.index[-1] - bothPrices.index[0]).total_seconds()/60/60/24
            
            print('Average frequency          :',  oneDayLTdata.reset_index().timestamp.diff(1).mean())
            print('Average trading size       :',  unit_size)
            print('Initial inventory          :',  initial_inventory_pool)
            print('Permanent impact param (L) :',  jump_size_L)
            print('Fill exponent              :',  fill_exponent)
            print('Price increment            :',  jump_size_L)
            
            pool_agent = get_pool_agent(arrival_rate, phi, alpha, fill_exponent, 
                                        initial_inventory_pool, target_inventory, 
                                        jump_size_L, unit_size, min_inventory_pool, max_inventory_pool,
                                        initial_price, max_depth,
                                        terminal_time, terminal_time/n_steps,
                                        seed, n_steps, num_trajectories, 
                                        eta_func_bid, eta_func_ask,
                                        verbose)
            print('pool agent:', pool_agent)
            
            ############################################
            # Run and store simulation result
            ############################################
            pool_earnings_history, arb_earnings_history, historical_pool_prices, historical_oracle_prices,\
            historical_ba, historical_pool_inventory, historical_pool_cash, historical_pool_value_adjustments\
                    = runOneSimulation(pool_agent, bothPrices, unit_size, 
                               min_inventory_pool, max_inventory_pool, target_inventory,
                               eta_func_bid, eta_func_ask, jump_size_L, 2)
            
            print('getting data')
            initial_pool_value = 0
            allData_, bid_history, ask_history = getOneSimulationData(initial_pool_value, initial_inventory_pool,
                                                                      historical_pool_prices,  historical_ba, 
                                                                      historical_pool_inventory, pool_earnings_history, 
                                                                      historical_oracle_prices, historical_pool_cash, 
                                                                      historical_pool_value_adjustments)


            uniswapindex  = allData_.reset_index().dropna().index
            allData       = allData_.dropna()
            bid_history   = bid_history.loc[allData.index]
            ask_history   = ask_history.loc[allData.index]
            allData.index = bothPrices.iloc[uniswapindex].index

            dict_results[to_datetime] = allData
        except Exception as e:
            print('ERROR : ', str(e))
        
        if to_datetime == f'{trade_date} 23:30': end_of_day = True
        from_dt     += datetime.timedelta(minutes = 30)
        to_dt       += datetime.timedelta(minutes = 30)
        from_dt_is  += datetime.timedelta(minutes = 30)